In [ ]:
# create empty database
from quantrocket.history import create_usstock_db
create_usstock_db("usstock-free-1d", bar_size="1 day", free=True)

In [ ]:
# populate the database with data (data is collected asynchronously)
from quantrocket.history import collect_history
collect_history("usstock-free-1d")

In [ ]:
# get securities
from quantrocket.master import get_securities
securities = get_securities(vendors="usstock", sec_types="STK")
securities.head()

In [ ]:
# create universe
from quantrocket.master import create_universe
create_universe("usstock-free", sids=securities.index.tolist())

In [ ]:
# filter securities from subset of securities
filtered_securities = securities[securities.Delisted==False]
filtered_securities = filtered_securities[["Symbol", "Exchange", "Name", "Delisted"]]
filtered_securities.head()

In [ ]:
# create universe from the filtered securities
create_universe("usstock-free-active", sids=filtered_securities.index.tolist())

In [11]:
# import pandas library for data manipulation
import pandas as pd

In [12]:
# download the aapl price stock data for apple (sid = aapl) from '01-01-2023' to '31-12-2023'
# parameters are db name, startDate, endDate, sid, required fields, filepath to store the downloaded file
# can specify the universe also
# downloaded file stored as aapl_prices.csv
from quantrocket.history import download_history_file
sid_aapl = "FIBBG000B9XRY4"
download_history_file(
    "usstock-free-1d",
    start_date="2023-01-01",
    end_date="2023-12-31",
    sids=sid_aapl,
    fields=["Close"],
    filepath_or_buffer="aapl_prices.csv"
)

In [19]:
# read the aapl_prices.csv file
df = pd.read_csv("aapl_prices.csv")

In [20]:
# Add the 's' column to the DataFrame and initialize with '0'
df['s'] = '0'

In [21]:
# Initialize variables
# Portfolio value
V = 0  
# Count of different states
state_counts = {'+1': 0, '0': 0, '-1': 0} 
# Transition counts 
transition_counts = {'+1': {'+1': 0, '0': 0, '-1': 0},
                     '0': {'+1': 0, '0': 0, '-1': 0},
                     '-1': {'+1': 0, '0': 0, '-1': 0}}
# List to store buy indices  
buy_indices = []  

In [22]:
# Iterate over the DataFrame rows
for i in range(1, len(df)):
    current_price = df.loc[i, 'Close']
    previous_price = df.loc[i - 1, 'Close']
    returns = (current_price - previous_price) / previous_price * 100  # Convert to percentage

    # Calculate the state
    if returns >= 0.1:
        state = '+1'
    elif returns > -0.1:
        state = '0'
    else:
        state = '-1'

    # Update the state counts
    state_counts[state] += 1

    # Calculate the portfolio value and track buy indices
    if state == '+1' and df.loc[i - 1, 's'] == '0':
        V += 1
        buy_indices.append(i)

    elif state == '-1' and df.loc[i - 1, 's'] == '0':
        V -= 1
        buy_indices.append(i)

    # Update the transition counts
    transition_counts[df.loc[i - 1, 's']][state] += 1

    # Update the 's' column in the DataFrame
    df.loc[i, 's'] = state

In [23]:
# Calculate the transition distribution
transition_distribution = {}
for from_state in transition_counts:
    total_transitions = sum(transition_counts[from_state].values())
    transition_distribution[from_state] = {
        to_state: transition_counts[from_state][to_state] / total_transitions
        for to_state in transition_counts[from_state]
    }

In [24]:
# Print the portfolio value, buy indices, and transition distribution
print('Portfolio Value:', V)
print('Buy Indices:', buy_indices)
print('Transition Distribution:')
for from_state in transition_distribution:
    print(from_state, '->', transition_distribution[from_state])

Portfolio Value: 6
Buy Indices: [1, 8, 12, 72, 79, 82, 87, 93, 96, 103, 116, 133, 153, 198, 203, 222, 227, 240, 245, 248]
Transition Distribution:
+1 -> {'+1': 0.5267175572519084, '0': 0.061068702290076333, '-1': 0.4122137404580153}
0 -> {'+1': 0.65, '0': 0.0, '-1': 0.35}
-1 -> {'+1': 0.5, '0': 0.11224489795918367, '-1': 0.3877551020408163}
